**Процедура стандартизации данных**

In [17]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.tools.tools import add_constant

# 1. Определение

Пусть имеется множество измерений определенного признака $x_i, i=\overline{1,n}$. Тогда стандартизацией такого ряда называется преобразование по формуле:

$$\tilde{x}_{i} = \frac{x_{i} - \bar{x}}{\sigma_x}. \tag{1.1}$$

Где:
- $\bar{x}$ - среднее арифметческое рассматртваемого ряда;
- $\sigma_x$ - стандартное отклонение.

Получется, что выражение $(1.1)$ может быть переписано следующим образом:

$$\tilde{x}_{i} = \frac{x_{i} - \bar{x}}{\sqrt{\frac{1}{n} \sum_{i=1}^n(x_i - \bar{x})^2}}.$$

Иногда предпочитают не отнимать среднее арифметическое в числителе, тогда формула $(1.1)$ принимает вид:

$$\tilde{x}_i  = \frac{x_i}{\sigma}.$$

# Свойсва результата

Величина получаемая в результате применения формулы $(1.1)$ получает следующие свойсва:

1. **Среднее артиaметисеское результата равняется нулю:**

$$\frac{\sum_{i=1}^n\tilde{x_i}}{n} = \frac{1}{n}\sum_{i=1}^n \frac{x_i - \bar{x}}{\sigma} = \frac{1}{n\sigma}\left[\sum_{i=1}^n x_i - \sum_{i=1}^n\bar{x}\right] = \frac{1}{n\sigma}\left[\sum_{i=1}^n x_i - n\bar{x} \right] =
\frac{1}{n\sigma}\left[ \sum_{i=1}^n x_i - n \sum_{i=1}^n\frac{x_i}{n} \right] = 0.$$

2. **Стандаратное отклонение равняется единице:**

Запишем стандартное отлонение $\tilde{x}_i$:

$$\sum_{i=1}^n \frac{\tilde{x}_i - \bar{\tilde{x}}_i}{n}$$

Но читвая результаты пункта 1 ($\bar{\tilde{x}}_i = 0$), получаем:


$$\sum_{i=1}^n \frac{\tilde{x}_i}{n} = 
\frac{1}{n}\sum_{i=1}^n\left[ \frac{x_i-\bar{x}}{\sqrt{\frac{1}{n} \sum_{i=1}^n (x_i - \bar{x})^2}} \right]^2 = 
\frac{1}{n}\sum_{i=1}^n\left[ \frac{(x_i-\bar{x})^2}{\frac{1}{n} \sum_{i=1}^n (x_i - \bar{x})^2} \right] = 
\frac{1}{n}\left[ \frac{\sum_{i=1}^n(x_i-\bar{x})^2}{\frac{1}{n} \sum_{i=1}^n (x_i - \bar{x})^2} \right] = \frac{n}{n} = 1.$$

# Влияние на коээфициенты моделей

## Линейная регрессия

Модель на исходнынх данных в матричных обозначениях примет вид:

$$\hat{y} = b X.$$

Где:
- $X$ - факторная матрица;
- $\hat{y}$ - вектор столбец предсказаний модели;
- $b$ - вектор строка оценк коэффициентов модели.

Тогда модель на стандартизированных данных примет вид:

$$\hat{y} = \tilde{b} \tilde{X}.$$

Где:
- $\tilde{X}$ - стандартизированная матрица предикоторов;
- $\tilde{b}$ - оценки коэффициентов полученные при использовании стантатицованной фактороной матрицы.

Уточним, что факторную матрицу можно переписать:
$$\tilde{X} = \left(\begin{array}\\
    x_{11}/\sigma_{x_1}&x_{12}/\sigma_{x_2} & \cdots & x_{1p}/\sigma_{x_p}\\
    x_{21}/\sigma_{x_1}&x_{22}/\sigma_{x_2} & \cdots & x_{2p}/\sigma_{x_p}\\
    \vdots & \vdots & \ddots & \vdots\\
    x_{n1}/\sigma_{x_1}&x_{n2}/\sigma_{x_2} & \cdots & x_{np}/\sigma_{x_p}\\
\end{array}\right)$$

Где:
- $p$ - число переменных модели;
- $\sigma_{x_j}$ - стандартное отклоненение $j$-й переменной.

Такая матрица раскладывается:

Рассмотрим линейную взаимосвязь без свободного члена.

In [19]:
n = 200
np.random.seed(15)

x = pd.DataFrame({
    "x1":np.random.normal(0, 0.3, n),
    "x2":np.random.normal(0, 3, n)
})

y = x["x1"]*3 + x["x2"]*2 + (np.random.rand(n)-0.5) + 3

In [51]:
np.dot(np.transpose(np_x), np_y)

array([[ 521.42148362],
       [  25.0424286 ],
       [3288.33815164]])

In [52]:
np_y = y.to_numpy().reshape([n,1])
np_x = add_constant(x).to_numpy()
np.dot(
    np.linalg.inv(
        np.dot(np.transpose(np_x), np_x)
    ),
    np.dot(np.transpose(np_x), np_y)
)

array([[2.95566332],
       [3.13373587],
       [2.00359775]])

Коэффициент при использовании исходных данных примет вид:

In [20]:
basic_model = LinearRegression(fit_intercept = False).fit(
    add_constant(x), y)
basic_model.coef_

array([2.95566332, 3.13373587, 2.00359775])

Коэффициент при использованнии стандатризированных данных примет вид:

In [23]:
# стандартизованная модель
x_stand = (x-x.mean())/np.std(x)

stand_model = LinearRegression(fit_intercept = False).fit(
    add_constant(x_stand), y
)

stand_model.coef_

array([2.60710742, 0.95731566, 5.8421477 ])

Или, приводя коэффициент к использованию на исходных данных.

In [28]:
stand_model.coef_/np.concatenate([[1], x.std().to_numpy()])

array([2.60710742, 3.12589172, 1.99858248])

Коэффициенты очень близки!